In [10]:
import mido 
from mido import MidiFile, Message, MidiTrack
import torch
import os
import numpy as np
import time
os.chdir('D:\engsci\year 3\CLASS\APS360\Project') #Sets current working directory!

In [11]:
#https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c  

In [26]:
def CountTracks(directory):          #Count files and tracks in folder
    trackCount = 0
    fileCount = 0
    for file in os.listdir(directory):
        if file.endswith(".midi"):
            fileCount += 1
            midiDir = MidiFile(directory+"/"+file)
            for track in midiDir.tracks: # Locates the tracks in a 'song.midi'
                trackCount += 1 # Counts tracks.
    print(fileCount+" files")
    print(trackCount+" tracks")

    
def PrintMessages(mid):                # print midi messages
    for i, track in enumerate(mid.tracks): # Iterates through tracks and also outputs the index.
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            print(msg)

            
def PrintSomeMessages(mid):             #print first 200 midi messages
    track = mid.tracks[0]
    for i,msg in enumerate(track):
        if i < 200:
            print(msg)

In [31]:
def Midi2Numpy(mid):                                #converts to numpy array removing non-note messages
    track = mid.tracks[0]                           #0th track only contains meta-messages, all notes on 1st track
    notes = np.empty([0,4])                         #4 because we're tracking: note, velocity,time and sustain.
    time = 0
    for msg in track:
        if msg.type == "note_on":                   # only count "note" messages - other inputs i.e. foot pedals are ignored
            #if msg[1] > 0: ?
            notes = np.append(notes,np.array([[msg.note, msg.velocity, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0
        if msg.type == "note_off":
            notes = np.append(notes,np.array([[msg.note, 0, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0
        else:
            time += msg.time                        #adjust time when removing other messages (time is length of note? start time of notes from start of song?)
    return notes


def NumpyGetSustain(notes):                       # Removes the zeroes
    for i, msg in enumerate(notes):
        if msg[1] > 0:                            # if velocity is not 0 (if note is being turned on)
            j = 1
            sustain = 0
            while msg[0] != notes[i+j][0]:        # while note values are different
                sustain += notes[i+j][2]          # Update sustain of ith note with i+jth note's time.
                j += 1                            #search for next message with same note i.e. message telling that note was released
            notes[i,3] = sustain + notes[i+j][2]  # once next appearance of note identified, updat ith note's sustain as time on until next note.
    time = 0
#     for i, msg in enumerate(notes):                #What is this doing?
#         if msg[1] > 0:                            # If velocity is not 0
#             notes[i,2] += time                    # Update the ith note's time with 0?
#             time = 0
#         else:
#             time += msg[2]                        #adjust time to account for removed zeros. The data is technically still encoded into the sustain value, so this section was removed.
    notes = notes[notes[:,1] > 0]                 #filter for notes with positive velocities (note presses)
    return notes

In [32]:
def cleanupMessages(mid):
    track = mid.tracks[1]
    track2 = []
    for msg in track:
        if msg.type == "note_on":
            track2.append(msg)
    mid.tracks[1] = track2

In [33]:
mid = MidiFile(r'D:\engsci\year 3\CLASS\APS360\Project\pivot\dataset\IRMAS-Sample\IRMAS-Sample\Training\vio\134__[vio][nod][cla]2156__3.wav.mid') #drummer data Has data on track 0... has .mid instead of .midi
#cleanupMessages(mid)
#mid.save("processed.midi")
PrintSomeMessages(mid) #prints first 200 msg in track 1 of mid.


<meta message set_tempo tempo=500000 time=0>
program_change channel=0 program=0 time=0
note_on channel=0 note=71 velocity=20 time=0
note_on channel=0 note=81 velocity=22 time=0
note_on channel=0 note=100 velocity=24 time=0
note_on channel=0 note=55 velocity=16 time=2
note_on channel=0 note=74 velocity=18 time=1
note_on channel=0 note=86 velocity=19 time=0
note_off channel=0 note=100 velocity=64 time=0
note_off channel=0 note=71 velocity=64 time=3
note_on channel=0 note=67 velocity=20 time=3
note_off channel=0 note=81 velocity=64 time=1
note_off channel=0 note=55 velocity=64 time=2
note_on channel=0 note=83 velocity=33 time=0
note_on channel=0 note=102 velocity=20 time=0
note_off channel=0 note=67 velocity=64 time=2
note_off channel=0 note=74 velocity=64 time=0
note_off channel=0 note=86 velocity=64 time=2
note_on channel=0 note=86 velocity=17 time=4
note_off channel=0 note=102 velocity=64 time=1
note_off channel=0 note=83 velocity=64 time=2
note_on channel=0 note=67 velocity=21 time=4


In [34]:
t0 = time.time()

mid = MidiFile(r'D:\engsci\year 3\CLASS\APS360\Project\pivot\dataset\IRMAS-Sample\IRMAS-Sample\Training\vio\134__[vio][nod][cla]2156__3.wav.mid')
print(type(mid))
notes = Midi2Numpy(mid)
print('before adding sustain: \n',notes[:20],'\n')
cleanNotes = NumpyGetSustain(notes)
print('after sustain is added: \n',cleanNotes[:20])
print(cleanNotes.shape)
np.savetxt("test.csv", cleanNotes, delimiter=",")

t1 = time.time()
print(t1-t0)
print('number of samples in array: ',notes.shape[0])
#print(notes) #Shouldn't first time be 28?

#Save the cleanNotes for testing:
import numpy
numpy.save('cleanNotes_1.npy',cleanNotes)
# print(cleanNotes[:,0].tolist())
# print(len(set(cleanNotes[:,2].tolist())))

<class 'mido.midifiles.midifiles.MidiFile'>
before adding sustain: 
 [[ 71.  20.   0.   0.]
 [ 81.  22.   0.   0.]
 [100.  24.   0.   0.]
 [ 55.  16.   2.   0.]
 [ 74.  18.   3.   0.]
 [ 86.  19.   1.   0.]
 [100.   0.   0.   0.]
 [ 71.   0.   3.   0.]
 [ 67.  20.   3.   0.]
 [ 81.   0.   4.   0.]
 [ 55.   0.   2.   0.]
 [ 83.  33.   0.   0.]
 [102.  20.   0.   0.]
 [ 67.   0.   2.   0.]
 [ 74.   0.   0.   0.]
 [ 86.   0.   2.   0.]
 [ 86.  17.   4.   0.]
 [102.   0.   5.   0.]
 [ 83.   0.   2.   0.]
 [ 67.  21.   4.   0.]] 

after sustain is added: 
 [[ 71.  20.   0.   9.]
 [ 81.  22.   0.  16.]
 [100.  24.   0.   6.]
 [ 55.  16.   2.  16.]
 [ 74.  18.   3.  15.]
 [ 86.  19.   1.  16.]
 [ 67.  20.   3.   8.]
 [ 83.  33.   0.  15.]
 [102.  20.   0.  13.]
 [ 86.  17.   4.  15.]
 [ 67.  21.   4.  11.]
 [ 79.  46.   2.  24.]
 [ 98.  45.   2.  17.]
 [ 81.  12.   6.  14.]
 [100.  21.   6.   3.]
 [100.  30.   1.   8.]
 [ 79.  24.   2.  23.]
 [ 98.  17.   2.  12.]
 [ 71.  21.   1.   7.]
 [ 90